# working pipeline 

In [1]:
import kfp
from kfp.dsl import pipeline, component
from datetime import datetime
from typing import NamedTuple

# Step 1: Fetch data from Trino and save it to MinIO
@component(
    base_image='bitnami/spark:3.5', 
    packages_to_install=['trino', 'pandas', 'pyarrow', 's3fs', 'boto3', 'urllib3']
)
def fetch_data_trino(minio_bucket: str) -> str:
    import trino
    from trino.auth import BasicAuthentication
    import csv
    import boto3
    import os
    import urllib3
    from datetime import datetime
    
    # Disable SSL warnings
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

    # Trino Connection Details
    TRINO_HOST = "192.168.80.155"
    TRINO_PORT = "30071"
    TRINO_USER = "ctzn.bank"
    TRINO_PASSWORD = "ctzn.bank_123"
    TRINO_CATALOG = "iceberg"
    TRINO_SCHEMA = "silver_crmuser"
    TRINO_HTTP_SCHEME = "https"

    # Output file
    OUTPUT_FILE = "/tmp/ctzn_10_years_data.csv"

    # SQL Query
    SQL_QUERY = """
    WITH recent_customers AS (
        SELECT DISTINCT g.cif_id
        FROM gold.dim_gam AS g
        WHERE CAST(DATE_PARSE(SUBSTRING(g.acct_opn_date, 1, 19), '%Y-%m-%dT%H:%i:%s') AS DATE) >= CURRENT_DATE - INTERVAL '10' YEAR
    ),
    account_activity AS (
        SELECT 
            a.cif_id,
            SUM(COALESCE(a.total_credit_tran_vol, 0) - COALESCE(a.total_debit_tran_vol, 0)) AS balance,
            COUNT(DISTINCT a.nepali_month)/6.0 AS balance_frequency,
            SUM(COALESCE(a.total_debit_tran_vol, 0)) AS purchases,
            MAX(COALESCE(a.total_debit_tran_vol, 0)) AS oneoff_purchases,
            SUM(COALESCE(a.total_debit_tran_vol, 0)) - MAX(COALESCE(a.total_debit_tran_vol, 0)) AS installments_purchases,
            SUM(CASE WHEN COALESCE(a.total_credit_tran_vol, 0) > 0 AND COALESCE(a.total_debit_tran_vol, 0) = 0 
                    THEN COALESCE(a.total_credit_tran_vol, 0) ELSE 0 END) AS cash_advance,
            COUNT(DISTINCT a.foracid)/6.0 AS purchases_frequency,
            COUNT(DISTINCT CASE WHEN COALESCE(a.total_debit_tran_vol, 0) > 100000 THEN a.foracid END)/6.0 AS oneoff_purchases_frequency,
            COUNT(DISTINCT CASE WHEN COALESCE(a.total_debit_tran_vol, 0) BETWEEN 1 AND 100000 THEN a.foracid END)/6.0 AS purchases_installments_frequency,
            COUNT(DISTINCT CASE WHEN COALESCE(a.total_credit_tran_vol, 0) > 0 THEN a.foracid END)/6.0 AS cash_advance_frequency,
            COUNT(DISTINCT CASE WHEN COALESCE(a.total_credit_tran_vol, 0) > 0 THEN a.foracid END) AS cash_advance_trx,
            COUNT(DISTINCT a.foracid) AS purchases_trx,
            SUM(COALESCE(a.total_credit_tran_vol, 0)) AS payments,
            COUNT(DISTINCT CASE WHEN COALESCE(a.total_credit_tran_vol, 0) >= COALESCE(a.total_debit_tran_vol, 0) 
                                THEN a.nepali_month END)/6.0 AS prc_full_payment
        FROM gold.mv_fact_deposit_account_insights a
        JOIN recent_customers rc ON a.cif_id = rc.cif_id
        GROUP BY a.cif_id
    ),
    salary_stats AS (
        SELECT 
            APPROX_PERCENTILE(COALESCE(salary_per_month, 0), 0.5) AS median_salary,
            APPROX_PERCENTILE(COALESCE(salary_per_month, 0), 0.05) AS fifth_percentile_salary
        FROM gold.dim_customers
    ),
    customer_profile AS (
        SELECT 
            g.cif_id,
            DATE_DIFF('year', 
                     CAST(DATE_PARSE(SUBSTRING(MIN(g.acct_opn_date), 1, 19), '%Y-%m-%dT%H:%i:%s') AS DATE), 
                     CURRENT_DATE) AS tenure,
            (SELECT fifth_percentile_salary FROM salary_stats) AS minimum_payments
        FROM gold.dim_gam g
        LEFT JOIN gold.dim_customers c ON g.cif_id = c.cif_id
        GROUP BY g.cif_id
    )
    SELECT 
        aa.cif_id AS custid,
        aa.balance,
        aa.balance_frequency,
        aa.purchases,
        aa.oneoff_purchases,
        aa.installments_purchases,
        aa.cash_advance,
        aa.purchases_frequency,
        aa.oneoff_purchases_frequency,
        aa.purchases_installments_frequency,
        aa.cash_advance_frequency,
        aa.cash_advance_trx,
        aa.purchases_trx,
        (SELECT median_salary * 3 FROM salary_stats) AS credit_limit,
        aa.payments,
        cp.minimum_payments,
        aa.prc_full_payment,
        cp.tenure
    FROM account_activity aa
    JOIN customer_profile cp ON aa.cif_id = cp.cif_id
    ORDER BY aa.cif_id
    """

    try:
        # Connect to Trino
        conn = trino.dbapi.connect(
            host=TRINO_HOST,
            port=TRINO_PORT,
            user=TRINO_USER,
            auth=BasicAuthentication(TRINO_USER, TRINO_PASSWORD),
            catalog=TRINO_CATALOG,
            schema=TRINO_SCHEMA,
            http_scheme=TRINO_HTTP_SCHEME,
            request_timeout=600,
            verify=False
        )
        cursor = conn.cursor()
        
        # Execute query and fetch data
        cursor.execute(SQL_QUERY)
        columns = [desc[0] for desc in cursor.description]
        
        # Write to CSV in batches
        with open(OUTPUT_FILE, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(columns)
            
            while True:
                rows = cursor.fetchmany(1000)
                if not rows:
                    break
                writer.writerows(rows)
        
        # Upload to MinIO
        minio_client = boto3.client(
            's3',
            endpoint_url="http://192.168.80.155:32000",
            aws_access_key_id="admin",
            aws_secret_access_key="dlyticaD123",
            verify=False
        )
        
        current_date = datetime.now().strftime("%Y-%m-%d")
        minio_path = f"data/customer_segmentation_raw_{current_date}.csv"
        minio_client.upload_file(OUTPUT_FILE, minio_bucket, minio_path)
        
        return minio_path
        
    except Exception as e:
        print(f"Error in fetch_data_trino: {str(e)}")
        raise
    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'conn' in locals():
            conn.close()


In [4]:
import kfp
from kfp.dsl import pipeline, component
from datetime import datetime
from typing import NamedTuple

# Step 1: Fetch data from Trino and save it to MinIO
@component(
    base_image='bitnami/spark:3.5', 
    packages_to_install=['trino', 'pandas', 'pyarrow', 's3fs', 'boto3', 'urllib3']
)
def fetch_data_trino(minio_bucket: str) -> str:
    import trino
    from trino.auth import BasicAuthentication
    import csv
    import boto3
    import os
    import urllib3
    from datetime import datetime
    
    # Disable SSL warnings
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

    # Trino Connection Details
    TRINO_HOST = "192.168.80.155"
    TRINO_PORT = "30071"
    TRINO_USER = "ctzn.bank"
    TRINO_PASSWORD = "ctzn.bank_123"
    TRINO_CATALOG = "iceberg"
    TRINO_SCHEMA = "gold"
    TRINO_HTTP_SCHEME = "https"

    # Output file
    OUTPUT_FILE = "/tmp/ctzn_10_years_data.csv"

    # SQL Query
    SQL_QUERY = """
    WITH recent_customers AS (
        SELECT DISTINCT g.cif_id
        FROM gold.dim_gam AS g
        WHERE CAST(DATE_PARSE(SUBSTRING(g.acct_opn_date, 1, 19), '%Y-%m-%dT%H:%i:%s') AS DATE) >= CURRENT_DATE - INTERVAL '10' YEAR
    ),
    account_activity AS (
        SELECT 
            a.cif_id,
            SUM(COALESCE(a.total_credit_tran_vol, 0) - COALESCE(a.total_debit_tran_vol, 0)) AS balance,
            COUNT(DISTINCT a.nepali_month)/6.0 AS balance_frequency,
            SUM(COALESCE(a.total_debit_tran_vol, 0)) AS purchases,
            MAX(COALESCE(a.total_debit_tran_vol, 0)) AS oneoff_purchases,
            SUM(COALESCE(a.total_debit_tran_vol, 0)) - MAX(COALESCE(a.total_debit_tran_vol, 0)) AS installments_purchases,
            SUM(CASE WHEN COALESCE(a.total_credit_tran_vol, 0) > 0 AND COALESCE(a.total_debit_tran_vol, 0) = 0 
                    THEN COALESCE(a.total_credit_tran_vol, 0) ELSE 0 END) AS cash_advance,
            COUNT(DISTINCT a.foracid)/6.0 AS purchases_frequency,
            COUNT(DISTINCT CASE WHEN COALESCE(a.total_debit_tran_vol, 0) > 100000 THEN a.foracid END)/6.0 AS oneoff_purchases_frequency,
            COUNT(DISTINCT CASE WHEN COALESCE(a.total_debit_tran_vol, 0) BETWEEN 1 AND 100000 THEN a.foracid END)/6.0 AS purchases_installments_frequency,
            COUNT(DISTINCT CASE WHEN COALESCE(a.total_credit_tran_vol, 0) > 0 THEN a.foracid END)/6.0 AS cash_advance_frequency,
            COUNT(DISTINCT CASE WHEN COALESCE(a.total_credit_tran_vol, 0) > 0 THEN a.foracid END) AS cash_advance_trx,
            COUNT(DISTINCT a.foracid) AS purchases_trx,
            SUM(COALESCE(a.total_credit_tran_vol, 0)) AS payments,
            COUNT(DISTINCT CASE WHEN COALESCE(a.total_credit_tran_vol, 0) >= COALESCE(a.total_debit_tran_vol, 0) 
                                THEN a.nepali_month END)/6.0 AS prc_full_payment
        FROM gold.mv_fact_deposit_account_insights a
        JOIN recent_customers rc ON a.cif_id = rc.cif_id
        GROUP BY a.cif_id
    ),
    salary_stats AS (
        SELECT 
            APPROX_PERCENTILE(COALESCE(salary_per_month, 0), 0.5) AS median_salary,
            APPROX_PERCENTILE(COALESCE(salary_per_month, 0), 0.05) AS fifth_percentile_salary
        FROM gold.dim_customers
    ),
    customer_profile AS (
        SELECT 
            g.cif_id,
            DATE_DIFF('year', 
                     CAST(DATE_PARSE(SUBSTRING(MIN(g.acct_opn_date), 1, 19), '%Y-%m-%dT%H:%i:%s') AS DATE), 
                     CURRENT_DATE) AS tenure,
            (SELECT fifth_percentile_salary FROM salary_stats) AS minimum_payments
        FROM gold.dim_gam g
        LEFT JOIN gold.dim_customers c ON g.cif_id = c.cif_id
        GROUP BY g.cif_id
    )
    SELECT 
        aa.cif_id AS custid,
        aa.balance,
        aa.balance_frequency,
        aa.purchases,
        aa.oneoff_purchases,
        aa.installments_purchases,
        aa.cash_advance,
        aa.purchases_frequency,
        aa.oneoff_purchases_frequency,
        aa.purchases_installments_frequency,
        aa.cash_advance_frequency,
        aa.cash_advance_trx,
        aa.purchases_trx,
        (SELECT median_salary * 3 FROM salary_stats) AS credit_limit,
        aa.payments,
        cp.minimum_payments,
        aa.prc_full_payment,
        cp.tenure
    FROM account_activity aa
    JOIN customer_profile cp ON aa.cif_id = cp.cif_id
    ORDER BY aa.cif_id
    """

    try:
        # Connect to Trino
        conn = trino.dbapi.connect(
            host=TRINO_HOST,
            port=TRINO_PORT,
            user=TRINO_USER,
            auth=BasicAuthentication(TRINO_USER, TRINO_PASSWORD),
            catalog=TRINO_CATALOG,
            schema=TRINO_SCHEMA,
            http_scheme=TRINO_HTTP_SCHEME,
            request_timeout=600,
            verify=False
        )
        cursor = conn.cursor()
        
        # Execute query and fetch data
        cursor.execute(SQL_QUERY)
        columns = [desc[0] for desc in cursor.description]
        
        # Write to CSV in batches
        with open(OUTPUT_FILE, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(columns)
            
            while True:
                rows = cursor.fetchmany(1000)
                if not rows:
                    break
                writer.writerows(rows)
        
        # Upload to MinIO
        minio_client = boto3.client(
            's3',
            endpoint_url="http://192.168.80.155:32000",
            aws_access_key_id="admin",
            aws_secret_access_key="dlyticaD123",
            verify=False
        )
        
        current_date = datetime.now().strftime("%Y-%m-%d")
        minio_path = f"data/customer_segmentation_raw_{current_date}.csv"
        minio_client.upload_file(OUTPUT_FILE, minio_bucket, minio_path)
        
        return minio_path
        
    except Exception as e:
        print(f"Error in fetch_data_trino: {str(e)}")
        raise
    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'conn' in locals():
            conn.close()

@component(
    base_image='quay.io/datanature_dev/jupyternotebook:java_home14',
    packages_to_install=[
        'pyspark==3.5.0',
        'pandas==2.0.3',
        'numpy==1.24.4',
        'boto3==1.28.57',
        'scikit-learn==1.3.0',
        'matplotlib==3.7.2',
        'pyarrow==12.0.1',
        'urllib3==2.0.4'
    ]
)
def feature_engineering_and_segmentation(
    file_path: str, 
    minio_bucket: str
) -> NamedTuple('Outputs', [
    ('segmentation_path', str),
    ('model_path', str),
    ('cluster_plot_path', str),
    ('cluster_interpretations', str),
    ('final_output_path', str)
]):
    # First ensure numpy is imported with the correct version
    import numpy as np
    np.__version__  # This helps catch version issues early
    
    # Then import other packages
    import boto3
    from botocore.config import Config
    import pandas as pd
    import matplotlib
    matplotlib.use('Agg')  # Set non-interactive backend
    import matplotlib.pyplot as plt
    from datetime import datetime
    import os
    import json
    import tempfile
    import time
    from collections import namedtuple
    from io import StringIO
    
    # Now import pyspark components
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import col, when, mean, monotonically_increasing_id, stddev, count, abs
    from pyspark.sql.types import DoubleType, StringType
    from pyspark.ml.feature import VectorAssembler, StandardScaler, PCA
    from pyspark.ml.clustering import BisectingKMeans
    from pyspark.ml.evaluation import ClusteringEvaluator
    from pyspark.sql.functions import udf

    def calculate_feature_distinctiveness(cluster_profiles):
        """Enhanced feature distinctiveness calculation"""
        feature_variance = cluster_profiles.std()
        distinctiveness = feature_variance / (feature_variance.max() + 1e-10)
        return distinctiveness

    def interpret_clusters(cluster_profiles):
        interpretations = {}
        distinctiveness = calculate_feature_distinctiveness(cluster_profiles)
        
        for cluster in cluster_profiles.index:
            features = cluster_profiles.columns.tolist()
            profile_components = []
            
            if 'purchases' in features:
                spend_level = cluster_profiles.loc[cluster, 'purchases']
                mean_spend = cluster_profiles['purchases'].mean()
                std_spend = cluster_profiles['purchases'].std()
                
                if spend_level > mean_spend + 1.5 * std_spend:
                    profile_components.append("High Spender")
                elif spend_level < mean_spend - 1.5 * std_spend:
                    profile_components.append("Low Spender")
                else:
                    profile_components.append("Moderate Spender")
            
            if 'cash_advance' in features and 'cash_advance_frequency' in features:
                cash_advance = cluster_profiles.loc[cluster, 'cash_advance']
                cash_freq = cluster_profiles.loc[cluster, 'cash_advance_frequency']
                mean_cash = cluster_profiles['cash_advance'].mean()
                mean_freq = cluster_profiles['cash_advance_frequency'].mean()
                
                if cash_advance > mean_cash * 2 and cash_freq > mean_freq * 2:
                    profile_components.append("Intensive Cash Advance User")
                elif cash_advance < mean_cash * 0.5 and cash_freq < mean_freq * 0.5:
                    profile_components.append("Rare Cash Advance User")
            
            if 'balance' in features and 'credit_limit' in features:
                balance_ratio = cluster_profiles.loc[cluster, 'balance'] / (cluster_profiles.loc[cluster, 'credit_limit'] + 1e-10)
                
                if balance_ratio > 0.8:
                    profile_components.append("Very High Credit Utilization")
                elif balance_ratio > 0.5:
                    profile_components.append("High Credit Utilization")
                elif balance_ratio < 0.2:
                    profile_components.append("Low Credit Utilization")
                else:
                    profile_components.append("Moderate Credit Utilization")
            
            if 'payments' in features and 'minimum_payments' in features:
                payment_ratio = cluster_profiles.loc[cluster, 'payments'] / (cluster_profiles.loc[cluster, 'minimum_payments'] + 1e-10)
                
                if payment_ratio > 3:
                    profile_components.append("Aggressive Overpayer")
                elif payment_ratio > 2:
                    profile_components.append("Consistent Overpayer")
                elif payment_ratio < 1.2:
                    profile_components.append("Minimum Payment User")
            
            if 'balance' in features and 'credit_limit' in features and 'cash_advance' in features:
                risk_score = (
                    cluster_profiles.loc[cluster, 'balance'] / (cluster_profiles.loc[cluster, 'credit_limit'] + 1e-10) * 0.4 +
                    cluster_profiles.loc[cluster, 'cash_advance'] / (cluster_profiles.loc[cluster, 'credit_limit'] + 1e-10) * 0.6
                )
                
                if risk_score > 0.7:
                    profile_components.append("Extremely High Financial Risk")
                elif risk_score > 0.5:
                    profile_components.append("High Financial Risk")
                elif risk_score < 0.2:
                    profile_components.append("Low Financial Risk")
                else:
                    profile_components.append("Moderate Financial Risk")
            
            if profile_components:
                interpretations[cluster] = "; ".join(profile_components)
            else:
                interpretations[cluster] = "Undefined Customer Segment"
        
        return interpretations

    def upload_to_minio(local_path, bucket, object_key, max_retries=5):
        """Uploads a file to MinIO with improved error handling and content-length issues fixed"""
        for attempt in range(max_retries):
            try:
                # Create a fresh client for each upload attempt
                client = boto3.client(
                    's3',
                    endpoint_url="http://192.168.80.155:32000",
                    aws_access_key_id="admin",
                    aws_secret_access_key="dlyticaD123",
                    verify=False,
                    config=Config(
                        connect_timeout=30,
                        read_timeout=60,
                        retries={'max_attempts': 3}
                    )
                )
                
                # Get the file size
                file_size = os.path.getsize(local_path)
                
                # For CSV files, we'll read and write with pandas to ensure proper formatting
                if object_key.endswith('.csv'):
                    df = pd.read_csv(local_path)
                    csv_buffer = StringIO()
                    df.to_csv(csv_buffer, index=False)
                    client.put_object(
                        Bucket=bucket,
                        Key=object_key,
                        Body=csv_buffer.getvalue(),
                        ContentType='text/csv'
                    )
                else:
                    # For non-CSV files, use standard upload
                    with open(local_path, 'rb') as file_data:
                        client.put_object(
                            Bucket=bucket,
                            Key=object_key,
                            Body=file_data,
                            ContentLength=file_size
                        )
                    
                print(f"Successfully uploaded {local_path} to {bucket}/{object_key}")
                return f"{bucket}/{object_key}"
                
            except Exception as e:
                if attempt < max_retries - 1:
                    wait_time = 2 ** attempt  # Exponential backoff
                    print(f"Upload attempt {attempt+1} failed: {str(e)}. Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
                else:
                    print(f"Upload failed after {max_retries} attempts: {str(e)}")
                    raise

    # Initialize Spark with optimized settings
    spark = SparkSession.builder \
        .appName("AdvancedCustomerSegmentation") \
        .config("spark.driver.memory", "8g") \
        .config("spark.executor.memory", "8g") \
        .config("spark.sql.shuffle.partitions", "200") \
        .config("spark.sql.execution.arrow.pyspark.enabled", "false") \
        .config("spark.executor.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true") \
        .config("spark.driver.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true") \
        .getOrCreate()

    # Initialize MinIO client with improved configuration
    boto_config = Config(
        connect_timeout=30,
        read_timeout=60,
        retries={'max_attempts': 3}
    )
    
    minio_client = boto3.client(
        's3',
        endpoint_url="http://192.168.80.155:32000",
        aws_access_key_id="admin",
        aws_secret_access_key="dlyticaD123",
        verify=False,
        config=boto_config
    )

    try:
        today = datetime.now().strftime('%Y%m%d_%H%M%S')
        local_path = "/tmp/raw_data.csv"
        
        # Download with retry logic
        max_retries = 3
        for attempt in range(max_retries):
            try:
                minio_client.download_file(minio_bucket, file_path, local_path)
                break
            except Exception as e:
                if attempt == max_retries - 1:
                    raise
                time.sleep(5)

        # Read data with explicit schema
        df = spark.read \
            .option("header", "true") \
            .option("inferSchema", "true") \
            .csv(local_path)

        if 'custid' in df.columns:
            df = df.withColumnRenamed("custid", "cif_id")

        # Feature engineering
        df = df.withColumn("balance_utilization_ratio", 
            when(col("credit_limit") != 0, col("balance") / col("credit_limit")).otherwise(0)
        )
        df = df.withColumn("cash_advance_intensity", 
            when(col("credit_limit") != 0, col("cash_advance") / col("credit_limit")).otherwise(0)
        )
        df = df.withColumn("payment_effort_ratio", 
            when(col("minimum_payments") != 0, col("payments") / col("minimum_payments")).otherwise(0)
        )
        df = df.withColumn("purchase_diversity", 
            abs(col("oneoff_purchases") - col("installments_purchases")) / 
            (abs(col("oneoff_purchases") + col("installments_purchases")) + 1e-10)
        )
        df = df.na.fill(0)

        numeric_cols = [
            'purchases', 'oneoff_purchases', 'installments_purchases', 
            'cash_advance', 'cash_advance_frequency', 
            'balance', 'credit_limit', 
            'payments', 'minimum_payments',
            'balance_utilization_ratio', 
            'cash_advance_intensity', 
            'payment_effort_ratio',
            'purchase_diversity'
        ]

        # Feature transformation
        assembler = VectorAssembler(inputCols=numeric_cols, outputCol="features")
        assembled = assembler.transform(df.drop("cif_id"))
        scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=True, withStd=True)
        scaled_data = scaler.fit(assembled).transform(assembled)

        # Dimensionality reduction
        pca = PCA(k=5, inputCol="scaled_features", outputCol="pca_features")
        pca_data = pca.fit(scaled_data).transform(scaled_data)

        # Clustering optimization
        evaluator = ClusteringEvaluator(featuresCol='pca_features')
        k_range = range(6, 15)
        k_metrics = []

        for k in k_range:
            kmeans = BisectingKMeans(k=k, seed=42, featuresCol="pca_features")
            model = kmeans.fit(pca_data)
            predictions = model.transform(pca_data)
            
            cost = model.summary.trainingCost if hasattr(model.summary, 'trainingCost') else 0
            silhouette = evaluator.evaluate(predictions)
            
            centers = model.clusterCenters()
            separation = np.mean([np.min(np.linalg.norm(centers[i] - centers[j])) 
                                  for i in range(len(centers)) 
                                  for j in range(i+1, len(centers))])
            
            k_metrics.append({
                'k': k,
                'cost': cost,
                'silhouette': silhouette,
                'separation': separation
            })

        # Determine optimal k
        k_metrics_df = pd.DataFrame(k_metrics)
        k_metrics_df['normalized_cost'] = (k_metrics_df['cost'] - k_metrics_df['cost'].min()) / (k_metrics_df['cost'].max() - k_metrics_df['cost'].min())
        k_metrics_df['normalized_silhouette'] = (k_metrics_df['silhouette'] - k_metrics_df['silhouette'].min()) / (k_metrics_df['silhouette'].max() - k_metrics_df['silhouette'].min())
        k_metrics_df['normalized_separation'] = (k_metrics_df['separation'] - k_metrics_df['separation'].min()) / (k_metrics_df['separation'].max() - k_metrics_df['separation'].min())
        
        k_metrics_df['composite_score'] = (
            0.3 * (1 - k_metrics_df['normalized_cost']) +
            0.4 * k_metrics_df['normalized_silhouette'] +
            0.3 * k_metrics_df['normalized_separation']
        )
        
        optimal_k = k_metrics_df.loc[k_metrics_df['composite_score'].idxmax(), 'k']

        # Plotting
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 15))
        ax1.plot(k_metrics_df['k'], k_metrics_df['cost'], marker='o', color='blue')
        ax1.set_xlabel('Number of Clusters (k)')
        ax1.set_ylabel('Cost', color='blue')
        ax1.grid(True)

        ax2.plot(k_metrics_df['k'], k_metrics_df['silhouette'], marker='x', color='green')
        ax2.set_xlabel('Number of Clusters (k)')
        ax2.set_ylabel('Silhouette Score', color='green')
        ax2.grid(True)

        ax3.plot(k_metrics_df['k'], k_metrics_df['separation'], marker='^', color='red')
        ax3.set_xlabel('Number of Clusters (k)')
        ax3.set_ylabel('Cluster Separation', color='red')
        ax3.grid(True)

        plt.tight_layout()
        cluster_plot_path = f"/tmp/cluster_diagnostics_{today}.png"
        plt.savefig(cluster_plot_path)
        plt.close()
        
        # Final clustering
        kmeans = BisectingKMeans(k=optimal_k, seed=42, featuresCol="pca_features")
        model = kmeans.fit(pca_data)
        clustered_data = model.transform(pca_data)

        # Add cluster information
        df_with_id = df.withColumn("row_id", monotonically_increasing_id())
        clustered_data = clustered_data.withColumn("row_id", monotonically_increasing_id())
        clustered_data = clustered_data.join(df_with_id.select("row_id", "cif_id"), on="row_id").drop("row_id")

        # Generate cluster profiles
        summary = clustered_data.groupBy("prediction").agg(
            *[mean(col(c)).alias(c) for c in numeric_cols],
            *[stddev(col(c)).alias(f"{c}_std") for c in numeric_cols],
            count("cif_id").alias("cluster_size")
        )
        
        # Convert to Pandas safely
        cluster_data = summary.collect()
        cluster_profiles = pd.DataFrame([row.asDict() for row in cluster_data]).set_index("prediction")
        
        # Get interpretations
        interpretations = interpret_clusters(cluster_profiles)
        
        # Add cluster interpretation to dataframe
        interpret_udf = udf(lambda x: interpretations.get(x, "Unknown"), StringType())
        clustered_data = clustered_data.withColumn("interpretation", interpret_udf(col("prediction")))

        # Prepare final output with all relevant columns
        output_columns = ["cif_id", "prediction", "interpretation"] + numeric_cols
        final_output = clustered_data.select(
            col("cif_id").alias("cif_id"),
            col("prediction").alias("cluster"),
            col("interpretation").alias("interpretations")
        )

        # Convert to Pandas DataFrame for CSV export
        final_pdf = pd.DataFrame(
            final_output.rdd.map(lambda row: row.asDict()).collect()
        )

        # Get current date in YYYY-MM-DD format
        current_date = datetime.now().strftime('%Y-%m-%d')
        
        # Define output paths
        output_paths = {
            'final_output': f"/tmp/final_segmented_customers_{current_date}.csv",
            'interpretations': f"/tmp/cluster_interpretations_{current_date}.json",
            'plot': f"/tmp/cluster_diagnostics_{current_date}.png",
            'model': f"/tmp/segmentation_model_{current_date}"
        }
        
        # Save final output (only the 3 columns)
        final_pdf.to_csv(output_paths['final_output'], index=False)
        print(f"Final output sample:\n{final_pdf.head()}")
        
        # Save interpretations
        with open(output_paths['interpretations'], 'w') as f:
            json.dump(interpretations, f, indent=2)
        
        # Save plot
        plt.savefig(output_paths['plot'])
        plt.close()
        
        # Save model
        model.write().overwrite().save(output_paths['model'])
        
        # Upload to MinIO with specified folder structure
        minio_paths = {}
        
        # Upload CSV files to results/
        csv_files = {
            'final_output': f"results/final_segmented_customers_{current_date}.csv",
            'interpretations': f"results/cluster_interpretations_{current_date}.json"
        }
        
        for name, minio_key in csv_files.items():
            try:
                uploaded_path = upload_to_minio(output_paths[name], minio_bucket, minio_key)
                minio_paths[name] = uploaded_path
            except Exception as e:
                print(f"Error uploading {name}: {str(e)}")
                raise

        # Upload plot to images/
        plot_key = f"images/cluster_diagnostics_{current_date}.png"
        try:
            uploaded_path = upload_to_minio(output_paths['plot'], minio_bucket, plot_key)
            minio_paths['plot'] = uploaded_path
        except Exception as e:
            print(f"Error uploading plot: {str(e)}")
            raise

        # Upload model to models/
        model_key_prefix = f"models/segmentation_model_{current_date}"
        try:
            for root, _, files in os.walk(output_paths['model']):
                for file in files:
                    file_path = os.path.join(root, file)
                    relative_path = os.path.relpath(file_path, output_paths['model'])
                    s3_key = f"{model_key_prefix}/{relative_path}"
                    upload_to_minio(file_path, minio_bucket, s3_key)
            minio_paths['model'] = f"{minio_bucket}/{model_key_prefix}"
        except Exception as e:
            print(f"Error uploading model files: {str(e)}")
            minio_paths['model'] = f"{minio_bucket}/{model_key_prefix}_partial"

        output = namedtuple('Outputs', [
            'segmentation_path',
            'model_path',
            'cluster_plot_path',
            'cluster_interpretations',
            'final_output_path'
        ])
        
        return output(
            f"{minio_bucket}/{csv_files['final_output']}",  # segmentation_path
            f"{minio_bucket}/{model_key_prefix}",           # model_path
            f"{minio_bucket}/{plot_key}",                   # cluster_plot_path
            f"{minio_bucket}/{csv_files['interpretations']}",  # cluster_interpretations
            f"{minio_bucket}/{csv_files['final_output']}"   # final_output_path
        )
        
    except Exception as e:
        print(f"Error in feature_engineering_and_segmentation: {str(e)}")
        raise
    finally:
        if spark:
            spark.stop()

@component(
    base_image='quay.io/datanature_dev/jupyternotebook:java_home14',
    packages_to_install=[
        'pyspark==3.5.0',
        'pandas==2.0.3',
        'numpy==1.24.4',
        'boto3==1.28.57',
        'pyarrow==12.0.1',
        'urllib3==2.0.4'
    ]
)
def save_predictions_to_trino(
    segmentation_path: str,
    minio_bucket: str
) -> str:
    import os
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import col
    from pyspark.sql.utils import AnalysisException

    # Configuration
    access_key_id = 'admin'
    secret_access_key = 'dlyticaD123'
    minio_endpoint = 'dn-minio-tenant-hl.dn-minio-tenant.svc.cluster.local:9000'
    data_bucket = 'ai360fd-recommendation'
    hive_metastore_uri = "thrift://dn-hive-metastore.dn-hive-metastore.svc.cluster.local:9083"
    iceberg_warehouse_location = f"s3a://{data_bucket}/data/"
    custom_catalog = "iceberg_catalog"
    app_name = "Customer Segmentation Loader"
    additional_packages = "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.2,org.apache.spark:spark-avro_2.12:3.5.0,com.crealytics:spark-excel_2.12:0.13.7,org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262,org.apache.hadoop:hadoop-common:3.3.4"
    
    # Initialize Spark Session with more compatible settings
    spark = SparkSession.builder \
        .appName(app_name) \
        .config("spark.driver.memory", "8g") \
        .config("spark.executor.memory", "8g") \
        .config("spark.executor.memoryOverhead", "2g") \
        .config("spark.driver.maxResultSize", "2g") \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .config("spark.hadoop.fs.s3a.endpoint", minio_endpoint) \
        .config("spark.hadoop.fs.s3a.access.key", access_key_id) \
        .config("spark.hadoop.fs.s3a.secret.key", secret_access_key) \
        .config("spark.hadoop.fs.s3a.path.style.access", "true") \
        .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
        .config("spark.hadoop.hive.metastore.uris", hive_metastore_uri) \
        .config("spark.sql.warehouse.dir", iceberg_warehouse_location) \
        .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog") \
        .config(f"spark.sql.catalog.{custom_catalog}", "org.apache.iceberg.spark.SparkCatalog") \
        .config(f"spark.sql.catalog.{custom_catalog}.warehouse", iceberg_warehouse_location) \
        .config(f"spark.sql.catalog.{custom_catalog}.s3.endpoint", minio_endpoint) \
        .config(f"spark.sql.catalog.{custom_catalog}.s3.access-key", access_key_id) \
        .config(f"spark.sql.catalog.{custom_catalog}.s3.secret-key", secret_access_key) \
        .config(f"spark.sql.catalog.{custom_catalog}.s3.path-style-access", "true") \
        .config("spark.jars.packages", additional_packages) \
        .config("spark.sql.repl.eagerEval.enabled", True) \
        .config("spark.sql.debug.maxToStringFields", 1000) \
        .config("spark.sql.legacy.createHiveTableByDefault", "false") \
        .config("spark.sql.iceberg.handle-timestamp-without-timezone", "true") \
        .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
        .config("spark.sql.shuffle.partitions", "10") \
        .config("spark.hadoop.fs.s3a.connection.maximum", "100") \
        .config("spark.sql.parquet.compression.codec", "snappy") \
        .enableHiveSupport() \
        .getOrCreate()

    try:
        # Extract the object key from the segmentation_path
        if segmentation_path.startswith(f"{minio_bucket}/"):
            object_key = segmentation_path[len(minio_bucket)+1:]
        else:
            object_key = segmentation_path
        
        # Construct the S3 path for Spark to read
        input_s3_path = f"s3a://{minio_bucket}/{object_key}"
        print(f"Reading data from: {input_s3_path}")
        
        # Read the segmentation data
        df = spark.read \
            .option("header", "true") \
            .option("inferSchema", "true") \
            .csv(input_s3_path)
        
        # Print input schema for debugging
        print("Input DataFrame Schema:")
        df.printSchema()
        
        # Rename columns to match our desired schema
        column_mapping = {
            "cluster": "cluster_id",
            "interpretations": "interpretation"  # Corrected spelling
        }
        
        for old_name, new_name in column_mapping.items():
            if old_name in df.columns:
                df = df.withColumnRenamed(old_name, new_name)
        
        # Ensure required columns exist
        required_columns = ["cif_id", "cluster_id", "interpretation"]
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            raise ValueError(f"Missing required columns: {missing_columns}")
        
        # Select only the required columns and cache to minimize re-computation
        df = df.select(*required_columns).cache()
        
        # Sample a small subset for testing if dataset is large
        # This helps with storage full issues - remove in production
        row_count = df.count()
        if row_count > 1000:
            print(f"Dataset has {row_count} rows, sampling for testing due to storage constraints")
            df = df.sample(fraction=1000.0/row_count).cache()
        
        # Define the table name
        table_name = f"{custom_catalog}.gold.customer_segments"
        temp_table_name = f"{custom_catalog}.gold.customer_segments_temp"
        
        # Handle existing table - this is crucial as we're getting an "already exists" error
        try:
            # Check if table exists by attempting to access it directly
            print(f"Checking if {table_name} exists")
            
            # Use more direct DataFrame API approach to check table existence
            tables = spark.catalog.listTables(f"{custom_catalog}.gold")
            table_exists = any(t.name.lower() == "customer_segments" for t in tables)
            
            if table_exists:
                print(f"Table {table_name} exists, will use 'overwrite' mode instead of 'create'")
                # Get sample of existing data to verify schema
                try:
                    existing_df = spark.table(table_name).limit(1)
                    print("Existing table schema:")
                    existing_df.printSchema()
                except Exception as e:
                    print(f"Could not read existing table: {str(e)}")
            else:
                print(f"Table {table_name} does not exist")
                
        except Exception as e:
            print(f"Error checking table existence: {str(e)}")
            print("Proceeding with assumption that table does not exist")
            table_exists = False
        
        # Write data - handle both cases (exists or not)
        print("Writing data to table...")
        try:
            if table_exists:
                # Try overwrite mode
                print(f"Using overwrite mode for existing table {table_name}")
                # df.writeTo(table_name).overwrite()

                df.writeTo(table_name).using("iceberg").partitionedBy("cluster_id").createOrReplace()

            else:
                # Try create mode with IF NOT EXISTS to avoid conflicts
                print(f"Creating new table {table_name}")
                try:
                    df.writeTo(table_name).using("iceberg").createOrReplace()
                except Exception as e:
                    # Last resort - try different approach if createOrReplace fails
                    print(f"Error with createOrReplace: {str(e)}")
                    print("Trying alternative approach with temporary table")
                    
                    # Create with temporary name first
                    df.writeTo(temp_table_name).using("iceberg").create()
                    
                    # Then rename or copy data if rename not supported
                    tables = spark.catalog.listTables(f"{custom_catalog}.gold")
                    if any(t.name.lower() == "customer_segments" for t in tables):
                        # If target exists, drop it first
                        spark.catalog.dropTable(table_name)
                    
                    # Now rename temp to target
                    spark._jsparkSession.catalog().renameTable(temp_table_name, table_name)
            
            # Verify the data was written
            verification_df = spark.table(table_name)
            count = verification_df.count()
            print(f"Successfully wrote {count} records to {table_name}")
            return f"Successfully loaded {count} records to {table_name}"
            
        except Exception as write_error:
            if "XMinioStorageFull" in str(write_error):
                print("Storage full error detected")
                # Handle storage full error by using compact format and reducing data size
                print("Attempting to write with more compact format and reduced dataset")
                
                # Sample smaller dataset as emergency fallback
                small_df = df.sample(fraction=0.1).cache()
                
                # Try write with most efficient settings
                small_df.coalesce(1).writeTo(table_name).using("iceberg").option("write-format", "parquet").option("compression", "snappy").createOrReplace()
                
                warn_msg = "WARNING: Storage was full - only wrote 10% sample of data"
                print(warn_msg)
                return warn_msg
            else:
                # Re-raise other errors
                raise
                
    except Exception as e:
        print(f"Error in save_predictions_to_trino: {str(e)}")
        return f"Error: {str(e)}"
    finally:
        if 'spark' in locals():
            spark.stop()

# Pipeline definition
@pipeline(name="Customer Segmentation Pipeline")
def customer_segmentation_pipeline(
    minio_bucket: str = "ai360ctzn-customer-segmentation"
):
    # Step 1: Fetch data
    fetch_task = fetch_data_trino(minio_bucket=minio_bucket)
    # fetch_task.set_caching_options(False)
    
    # Step 2: Process data and create segments
    segmentation_task = feature_engineering_and_segmentation(
        file_path=fetch_task.output,
        minio_bucket=minio_bucket
    )
    # segmentation_task.set_caching_options(False)
    
    # Step 3: Save results to Trino
    save_to_trino = save_predictions_to_trino(
        segmentation_path=segmentation_task.outputs['final_output_path'],
        minio_bucket=minio_bucket
    )
    save_to_trino.set_caching_options(False)

    

    save_to_trino.set_caching_options(False)
# Compile the pipeline
if __name__ == "__main__":
    from kfp import compiler
    compiler.Compiler().compile(
        customer_segmentation_pipeline,
        "customer_segmentation_pipeline.yaml"
    )
    print("Pipeline compiled successfully.")

In [ ]:
# Step 3: Save predictions to Trino
@component(
    base_image='quay.io/datanature_dev/jupyternotebook:java_home14',
    packages_to_install=['trino', 'pandas', 'boto3', 'urllib3']
)
def save_predictions_to_trino(
    segmentation_path: str,
    minio_bucket: str
) -> str:
    import trino
    from trino.auth import BasicAuthentication
    import pandas as pd
    import boto3
    import urllib3
    import pandas as pd
    from pyspark.sql import SparkSession
    from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
    from pyspark.ml import Pipeline
    from pyspark.ml.classification import LogisticRegression
    from pyspark.ml.evaluation import BinaryClassificationEvaluator
    import os
    import shutil
    from datetime import datetime
    from pyspark.sql.functions import lit, col, when
    
    # variables
    # variables
    # access_key_id = 'admin'
    # secret_access_key = 'dlyticaD123'
    # minio_endpoint = 'dn-minio-tenant-hl.dn-minio-tenant.svc.cluster.local:9000'  # Replace with your MinIO server endpoint
    # data_bucket = 'ai360fd-recommendation'
    # apps_bucket = ''
    # hive_metastore_uri = "thrift://dn-hive-metastore.dn-hive-metastore.svc.cluster.local:9083"
    # iceberg_warehouse_location = f"s3a://{data_bucket}/data/"
    # custom_catalog = "iceberg_catalog"
    # spark_eventlog_dir = f"s3a://{apps_bucket}/logs/spark/"
    # master_url="spark://dn-spark-master-svc.dn-spark.svc.cluster.local:7077"
    # spark_driver_host="notebook-five-spark-driver-headless.kubeflow-user-example-com.svc.cluster.local"
    # app_name="All Data Generation"
    # additional_packages="org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.2,org.apache.spark:spark-avro_2.12:3.5.0,com.crealytics:spark-excel_2.12:0.13.7,org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262,org.apache.hadoop:hadoop-common:3.3.4"

    
    access_key_id = 'admin'
    secret_access_key = 'dlyticaD123'
    minio_endpoint = 'dn-minio-tenant-hl.dn-minio-tenant.svc.cluster.local:9000'  # Replace with your MinIO server endpoint
    data_bucket = 'ai360fd-recommendation'
    # apps_bucket = 'dn-apps'
    hive_metastore_uri = "thrift://dn-hive-metastore.dn-hive-metastore.svc.cluster.local:9083"
    iceberg_warehouse_location = f"s3a://{data_bucket}/data/"
    custom_catalog = "iceberg_catalog"
    # spark_eventlog_dir = f"s3a://{apps_bucket}/logs/spark/"
    # master_url="spark://dn-spark-master-svc.dn-spark.svc.cluster.local:7077"
    # spark_driver_host="notebook-five-spark-driver-headless.kubeflow-user-example-com.svc.cluster.local"
    app_name="All Data Generation"
    additional_packages="org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.2,org.apache.spark:spark-avro_2.12:3.5.0,com.crealytics:spark-excel_2.12:0.13.7,org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262,org.apache.hadoop:hadoop-common:3.3.4"

    #spark configuration
    spark = SparkSession.builder \
        .appName(app_name) \
        .config("spark.driver.memory", "8g") \
        .config("spark.executor.memory", "8g") \
        .config("spark.executor.memoryOverhead", "2g") \
        .config("spark.driver.maxResultSize", "2g") \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .config("spark.hadoop.fs.s3a.endpoint", minio_endpoint) \
        .config("spark.hadoop.fs.s3a.access.key", access_key_id) \
        .config("spark.hadoop.fs.s3a.secret.key", secret_access_key) \
        .config("spark.hadoop.fs.s3a.path.style.access", "true") \
        .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
        .config("spark.hadoop.hive.metastore.uris", hive_metastore_uri) \
        .config("spark.sql.warehouse.dir", iceberg_warehouse_location) \
        .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog") \
        .config(f"spark.sql.catalog.{custom_catalog}", "org.apache.iceberg.spark.SparkCatalog") \
        .config("spark.jars.packages", additional_packages) \
        .config("spark.sql.repl.eagerEval.enabled", True) \
        .config("spark.sql.debug.maxToStringFields", 1000) \
        .enableHiveSupport() \
        .getOrCreate()
    
    # spark.sql(f"""
    # show tables in {custom_catalog}.gold
    # """).show(100,False)
    # print("train_model is called")
    # Initialize Spark session
    # spark = SparkSession.builder.appName("Feature Engineering").getOrCreate()
    
        
    # Disable SSL warnings
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

    try:
        # Download predictions from MinIO
        minio_client = boto3.client(
            's3',
            endpoint_url="http://192.168.80.155:32000",
            aws_access_key_id="admin",
            aws_secret_access_key="dlyticaD123",
            verify=False
        )
        
        local_path = "/tmp/predictions.csv"
        minio_client.download_file(minio_bucket, segmentation_path, local_path)
        predictions = pd.read_csv(local_path)
        
        # # Connect to Trino
        # conn = trino.dbapi.connect(
        #     host="192.168.80.155",
        #     port=30071,
        #     user="ctzn.bank",
        #     auth=BasicAuthentication("ctzn.bank", "ctzn.bank_123"),
        #     catalog="iceberg",
        #     schema="silver_crmuser",
        #     http_scheme="https",
        #     verify=False
        # )
        # cursor = conn.cursor()

        # # Create table if not exists
        # cursor.execute("""
        # CREATE TABLE IF NOT EXISTS iceberg.silver_crmuser.customer_segments (
        #     cif_id VARCHAR,
        #     cluster_id INTEGER,
        #     processed_date TIMESTAMP
        # ) WITH (
        #     partitioning = ARRAY['processed_date']
        # )
        # """)

        # Insert predictions in batches
        batch_size = 1000
        for i in range(0, len(predictions), batch_size):
            batch = predictions[i:i+batch_size]
            values = [(str(row['custid']), int(row['prediction'])) for _, row in batch.iterrows()]
            
            insert_sql = """
            INSERT INTO iceberg.silver_crmuser.customer_segments 
            (cif_id, cluster_id, processed_date) 
            VALUES (?, ?, CURRENT_TIMESTAMP)
            """
            
            cursor.executemany(insert_sql, values)
            conn.commit()

        return f"Successfully saved {len(predictions)} predictions to Trino"
        
    except Exception as e:
        print(f"Error in save_predictions_to_trino: {str(e)}")
        raise
    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'conn' in locals():
            conn.close()

# Pipeline definition
@pipeline(name="Customer Segmentation Pipeline")
def customer_segmentation_pipeline(
    minio_bucket: str = "ai360ctzn-customer-segmentation"
):
    # Step 1: Fetch data
    fetch_task = fetch_data_trino(minio_bucket=minio_bucket)
    
    # Step 2: Process data and create segments
    segmentation_task = feature_engineering_and_segmentation(
        file_path=fetch_task.output,
        minio_bucket=minio_bucket
    )
    
    # Step 3: Save results to Trino
    save_task = save_predictions_to_trino(
        segmentation_path=segmentation_task.outputs['segmentation_path'],
        minio_bucket=minio_bucket
    )

# Compile the pipeline
if __name__ == "__main__":
    from kfp import compiler
    compiler.Compiler().compile(
        customer_segmentation_pipeline,
        "customer_segmentation_pipeline.yaml"
    )
    print("Pipeline compiled successfully.")